# Examples for QMSim.jl

This demonstrates the basic functionality of `QMSim.jl` from matrix construction all the way to solving real quantum mechanics problems.

In [1]:
# activate the examples environment
import Pkg;
Pkg.activate(@__DIR__)
Pkg.add("Arpack")
# Pkg.resolve()
# Pkg.instantiate()
# Pkg.precompile()
# Pkg.update()

  Activating project at `~/Julia/QMSim.jl/examples`
   Resolving package versions...
  No Changes to `~/Julia/QMSim.jl/examples/Project.toml`
  No Changes to `~/Julia/QMSim.jl/examples/Manifest.toml`
Precompiling project...
   1578.6 ms  ✓ QMSim
  1 dependency successfully precompiled in 2 seconds. 116 already precompiled.


In [30]:
using Revise
using Profile
using SparseArrays, LinearAlgebra

import QGas.NumericalTools.ArrayDimensions as AD

using QMSim

## Demonstrate the overall setup of a matrix

This does not solve any problems.

In [3]:
# create an array of physical dimensions for our quantum system to live in, including an example with two space and one spin dimensions

dims = AD.Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5, unit="X Momentum", periodic=true, spatial=true),
    # DimensionWithSpace(; x0=-1.0, dx=1.0, npnts=3, unit="Spin", periodic=true, spatial=true),
)

"""
    build_rules!(mwr::MatrixWithRules)

add required rules to the matrix
"""
function build_rules!(mwr::MatrixWithRules)

    function ham_tunneling(x, y; J=1.0)
        return -J
    end

    add_rule!(mwr, RelativeRule, ham_tunneling, [1,])
    add_rule!(mwr, RelativeRule, ham_tunneling, [-1,])
    add_rule!(mwr, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))
    return mwr
end

# See how we define the type of the matrix we want!
mwr = MatrixWithRules(SparseMatrixCSC{ComplexF64}, dims)

build_rules!(mwr)

generate_builders!(mwr)

build!(mwr; J=2.0)

mwr

5×5 MatrixWithRules{ComplexF64, SparseMatrixCSC{ComplexF64}}:
  1.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im  -2.0+0.0im   3.0+0.0im  -2.0+0.0im   0.0+0.0im
  0.0+0.0im   0.0+0.0im  -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im   5.0+0.0im

Now we do the same thing, but for more than one matrix:

In [4]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_matrix!(mwrs, :tunneling)
add_matrix!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

generate_builders!(mwrs)

build!(mwrs; J=2.0)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -2.0+0.0im       ⋅           ⋅      -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im       ⋅           ⋅    
      ⋅      -2.0+0.0im   3.0+0.0im  -2.0+0.0im       ⋅    
      ⋅           ⋅      -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im       ⋅           ⋅      -2.0+0.0im   5.0+0.0im

In [5]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_matrix!(mwrs, :tunneling)
add_matrix!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

generate_builders!(mwrs)

build!(mwrs; J=2.0)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -2.0+0.0im       ⋅           ⋅      -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im       ⋅           ⋅    
      ⋅      -2.0+0.0im   3.0+0.0im  -2.0+0.0im       ⋅    
      ⋅           ⋅      -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im       ⋅           ⋅      -2.0+0.0im   5.0+0.0im

At this point this is not all that useful, but the main point is that with slightly more syntactic sugar we can write code to define pretty much any physics problem!

## Solve specific problems

The existing code already encapsulates any abstract matrix.  There is some work required to make this work efficiently for both dense and sparse matrices.

In [28]:
dims = AD.Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5001, unit="X Momentum", periodic=true, spatial=true),
    # DimensionWithSpace(; x0=-1.0, dx=1.0, npnts=3, unit="Spin", periodic=true, spatial=true),
)

# wms = QMSolver(SparseMatrixCSC{ComplexF64}, dims; num_states=6, wrap=Hermitian)
wms = QMSolver(Matrix{ComplexF64}, dims; num_states=6, wrap=Hermitian)

add_matrix!(wms, :tunneling)
add_matrix!(wms, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(wms, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(wms, :tunneling, RelativeRule, ham_tunneling, [-1,])

generate_builders!(wms)

build!(wms; J=2.0)

5001×5001 QMSolver{ComplexF64, Matrix{ComplexF64}}:
  0.0+0.0im  -2.0+0.0im   0.0+0.0im  …   0.0+0.0im   0.0+0.0im  -2.0+0.0im
 -2.0-0.0im   0.0+0.0im  -2.0+0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im  -2.0-0.0im   0.0+0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im  -2.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im  …   0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
     ⋮                               ⋱                              ⋮
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0-0.0im   0.0-0.0im   0.0-0.0im      0.0+0.0im   

In [ ]:
eigensystem!(wms; J=2.0)
wms.eigenvalues

5001-element Vector{ComplexF64}:
                -4.0 + 0.0im
 -3.9999968429899377 + 0.0im
 -3.9999968429899373 + 0.0im
  -3.999987371964733 + 0.0im
 -3.9999873719647328 + 0.0im
 -3.9999715869393366 + 0.0im
 -3.9999715869393366 + 0.0im
  -3.999949487938666 + 0.0im
 -3.9999494879386654 + 0.0im
  -3.999921074997603 + 0.0im
                     ⋮
  3.9999360707081135 + 0.0im
  3.9999613266839638 + 0.0im
  3.9999613266839638 + 0.0im
  3.9999802687007353 + 0.0im
  3.9999802687007353 + 0.0im
   3.999992896728527 + 0.0im
   3.999992896728527 + 0.0im
  3.9999992107474065 + 0.0im
  3.9999992107474065 + 0.0im